# Нейронные сети
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW06] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

Для начала вам предстоит реализовать свой собственный backpropagation и протестировать его на реальных данных, а затем научиться обучать нейронные сети при помощи библиотеки `PyTorch` и использовать это умение для классификации классического набора данных CIFAR10.

In [ ]:
import numpy as np
import copy
from sklearn.datasets import make_blobs, make_moons

In [ ]:
np.random.seed(9)

### Задание 1 (3 балла)
Нейронные сети состоят из слоев, поэтому для начала понадобится реализовать их. Пока нам понадобятся только три:

`Linear` - полносвязный слой, в котором `y = Wx + b`, где `y` - выход, `x` - вход, `W` - матрица весов, а `b` - смещение. 

`ReLU` - слой, соответствующий функции активации `y = max(0, x)`.

`Softmax` - слой, соответствующий функции активации [softmax](https://ru.wikipedia.org/wiki/Softmax)


#### Методы
`forward(X)` - возвращает предсказанные для `X`. `X` может быть как вектором, так и батчем

`backward(d)` - считает градиент при помощи обратного распространения ошибки. Возвращает новое значение `d`

`update(alpha)` - обновляет веса (если необходимо) с заданой скоростью обучения

In [ ]:
class Module:
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, d):
        raise NotImplementedError()
        
    def update(self, alpha):
        pass
        
class Linear(Module):
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weight = np.random.normal(scale = 0.5, size = (in_features, out_features))
        self.b = np.zeros(out_features)
    
    def forward(self, x):
        self.X = np.copy(x)
        self.z = x@self.weight + self.b
        return self.z
    
    def backward(self, d):
        n = self.X.shape[0]
        self.W_grad = (self.X).T@d
        self.b_grad = np.sum(d, axis = 0)
        return d@(self.weight).T
        
    def update(self, alpha):
        self.weight -= alpha*self.W_grad
        self.b -= alpha*self.b_grad 
        
class ReLU(Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        self.neg = x < 0
        z = np.copy(x)
        z[z < 0] = 0
        return z
        
    def backward(self, d):
        d_new = np.copy(d)
        d_new[self.neg] = 0
        return d_new
        
class Softmax(Module):
    def __init__(self):
        pass
        
    def forward(self, x):
        exp = np.exp(x - np.max(x, axis = 1, keepdims = True))
        s = np.sum(exp, axis = 1, keepdims = True)
        self.prob = exp/s
        return self.prob
        
    def backward(self, d):
        p = self.prob
        n = len(p)
        p[range(n), d] -= 1
        p = p/n
        return p        
        
        

### Задание 2 (2 балла)
Теперь сделаем саму нейронную сеть.

#### Методы
`fit(X, y)` - обучает нейронную сеть заданное число эпох. В каждой эпохе необходимо использовать [cross-entropy loss](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy) для обучения, а так же производить обновления не по одному элементу, а используя батчи.

`predict_proba(X)` - предсказывает вероятности классов для элементов `X`

#### Параметры конструктора
`modules` - список, состоящий из ранее реализованных модулей и описывающий слои нейронной сети. В конец необходимо добавить `Softmax`

`epochs` - количество эпох обучения

`alpha` - скорость обучения

In [ ]:
class MLPClassifier:
    def __init__(self, modules, epochs=40, alpha=0.01, batch = 10):
        self.modules = modules
        self.modules.append(Softmax())
        self.epochs = epochs
        self.alpha = alpha
        self.batch = batch
            
    def fit(self, X, y):
        y = np.array(y)
        n = X.shape[0]
        batch = self.batch
        for i in range(self.epochs):
            
            p = np.random.permutation(n)
    
            X_perm = X[p, :]
            y_perm = y[p]
            ind = 0
            probs = np.zeros((n, len(np.unique(y))) )
            while ind < n:
                
                X_batch = X_perm[ind:ind + batch, :]
                y_batch = y_perm[ind:ind + batch]  
                
                a = self.forward(X_batch)
                probs[ind : ind + batch, :] = a
                self.backward(y_batch)
                ind = ind + batch
                
            cost = self.cross_entropy_loss(probs, y_perm)
            
        print('cost:', cost)
                
    def forward(self, X):
        a = X
        for mod in self.modules:
            a = mod.forward(a)
        return a
    
    def backward(self, y):
        d = y
        for mod in reversed(self.modules):
            d = mod.backward(d)
            if isinstance(mod, Linear):
                        mod.update(self.alpha)
            
    def cross_entropy_loss(self, probs, y):
        n = probs.shape[0]
        log = -np.log(probs[range(n), y])
        cost = np.sum(log) / n
        return cost
        
    def predict_proba(self, X):
        return self.forward(X)

    def predict(self, X):
        p = self.predict_proba(X)
        return np.argmax(p, axis=1)


In [ ]:
p = MLPClassifier([
    Linear(4, 64),
    ReLU(),
    Linear(64, 64),
    ReLU(),
    Linear(64, 2)
], epochs = 100, alpha=0.025)

X = np.random.randn(50, 4)
y = [(0 if x[0] > x[2]**2 or x[3]**3 > 0.5 else 1) for x in X]
p.fit(X, y)
print(p.predict(X))
print(np.mean(p.predict(X) == y))

### Задание 3 (2 балла)
Протестируем наше решение на синтетических данных. Необходимо подобрать гиперпараметры, при которых качество полученных классификаторов будет достаточным.

#### Оценка
Accuracy на первом датасете больше 0.85 - +1 балл

Accuracy на втором датасете больше 0.85 - +1 балл

In [ ]:
X, y = make_moons(400, noise=0.075)
X_test, y_test = make_moons(400, noise=0.075)

best_acc = 0

layer = [
    [X.shape[1], 2, 3, 4, 5, 2],
    [X.shape[1], 4, 8, 4, 2],
    [X.shape[1], 64, 64, 2],
    [X.shape[1], 32, 32, 2],
    [X.shape[1], 4, 4, 4, 2]
]
for size in layer:
    modules = []
    for l in range(0, len(size)-1):
        modules.append(Linear(size[l], size[l+1]))
        if l < len(size)-2:
            modules.append(ReLU())
    p = MLPClassifier(modules, epochs = 50, alpha= 0.2)
    print('----------')
    p.fit(X, y)
    acc = np.mean(p.predict(X_test) == y_test)

    print(acc, 'размер слоев', size)
    best_acc = max(acc, best_acc)
print("Accuracy", best_acc)



In [ ]:
p = MLPClassifier([
    Linear(2, 64),
    ReLU(),
    Linear(64, 64),
    ReLU(),
    Linear(64, 2)
], epochs = 50, alpha=0.2)
p.fit(X, y)
acc = np.mean(p.predict(X_test) == y_test)
print("Accuracy", acc)

In [ ]:
X, y = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5], [-2.5, 3]])
X_test, y_test = make_blobs(400, 2, centers=[[0, 0], [2.5, 2.5], [-2.5, 3]])
best_acc = 0
layer = [
    [X.shape[1], 2, 3, 4, 5, 3],
    [X.shape[1], 4, 8, 4, 3],
    [X.shape[1], 64, 64, 3],
    [X.shape[1], 32, 32, 3],
    [X.shape[1], 4, 4, 4, 3]
]
for size in layer:
    modules = []

    for l in range(0, len(size)-1):
        modules.append(Linear(size[l], size[l+1]))
        if l < len(size)-2:
            modules.append(ReLU())
    p = MLPClassifier(modules, epochs = 100, alpha= 0.2)
    print('----------')
    p.fit(X, y)
    acc = np.mean(p.predict(X_test) == y_test)

    print(acc, 'размер слоев', size)

    best_acc = max(acc, best_acc)
print("Accuracy", best_acc)



In [ ]:
p = MLPClassifier([
    Linear(2, 64),
    ReLU(),
    Linear(64, 64),
    ReLU(),
    Linear(64, 3)
], epochs = 50, alpha=0.2)
p.fit(X, y)
acc = np.mean(p.predict(X_test) == y_test)
print("Accuracy", acc)

## PyTorch

Для выполнения следующего задания понадобится PyTorch. [Инструкция по установке](https://pytorch.org/get-started/locally/)

Если у вас нет GPU, то можно использовать [Google Colab](https://colab.research.google.com/)

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t = transforms.ToTensor()

cifar_train = datasets.CIFAR10("datasets/cifar10", download=True, train=True, transform=t)
train_loader = DataLoader(cifar_train, batch_size=1024, shuffle=True, pin_memory=torch.cuda.is_available())
cifar_test = datasets.CIFAR10("datasets/cifar10", download=True, train=False, transform=t)
test_loader = DataLoader(cifar_test, batch_size=1024, shuffle=False, pin_memory=torch.cuda.is_available())

Files already downloaded and verified
Files already downloaded and verified


In [12]:
def outputSize(in_size, kernel_size, stride, padding):
    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
    return(output)

### Задание 4 (3 балла)
А теперь поработаем с настоящими нейронными сетями и настоящими данными. Необходимо реализовать сверточную нейронную сеть, которая будет классифицировать изображения из датасета CIFAR10. Имплементируйте класс `Model` и функцию `calculate_loss`. 

Обратите внимание, что `Model` должна считать в конце `softmax`, т.к. мы решаем задачу классификации. Соответствено, функция `calculate_loss` считает cross-entropy.

Для успешного выполнения задания необходимо, чтобы `accuracy`, `mean precision` и `mean recall` были больше 0.5

__Можно пользоваться всем содержимым библиотеки PyTorch.__

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.linear1 = nn.Linear(16 * 6 * 6, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
        
criterion = nn.CrossEntropyLoss()
def calculate_loss(X, y, model):
    return criterion(model(X), y)

Теперь обучим нашу модель. Для этого используем ранее созданные batch loader'ы.

In [7]:
def train(model, epochs = 100):
    optimizer = torch.optim.Adam(model.parameters())
    train_losses = []
    test_losses = []
    for i in range(epochs):
        #Train
        loss_mean = 0
        elements = 0
        for X, y in iter(train_loader):
            X = X.to(device)
            y = y.to(device)
            loss = calculate_loss(X, y, model)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_mean += loss.item() * len(X)
            elements += len(X)
        train_losses.append(loss_mean / elements)
        #Test
        loss_mean = 0 
        elements = 0
        for X, y in iter(test_loader):
            X = X.to(device)
            y = y.to(device)
            loss = calculate_loss(X, y, model)
            loss_mean += loss.item() * len(X)
            elements += len(X)
        test_losses.append(loss_mean / elements)
        print("Epoch", i, "| Train loss", train_losses[-1], "| Test loss", test_losses[-1])
    return train_losses, test_losses

In [8]:
model = Model().to(device)
train_l, test_l = train(model)

tensor([[ 0.0323, -0.0323, -0.0848,  ..., -0.0114,  0.0340,  0.0602],
        [ 0.0316, -0.0331, -0.0808,  ..., -0.0125,  0.0314,  0.0521],
        [ 0.0309, -0.0314, -0.0768,  ..., -0.0130,  0.0310,  0.0549],
        ...,
        [ 0.0355, -0.0330, -0.0703,  ..., -0.0165,  0.0309,  0.0635],
        [ 0.0269, -0.0295, -0.0897,  ..., -0.0095,  0.0357,  0.0469],
        [ 0.0294, -0.0265, -0.0775,  ..., -0.0158,  0.0364,  0.0551]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([6, 9, 4,  ..., 7, 5, 7], device='cuda:0')
tensor([[ 0.0355, -0.0107, -0.0536,  ...,  0.0156,  0.0372,  0.0830],
        [ 0.0293, -0.0145, -0.0408,  ...,  0.0150,  0.0352,  0.0746],
        [ 0.0277, -0.0212, -0.0633,  ...,  0.0093,  0.0331,  0.0742],
        ...,
        [ 0.0315, -0.0195, -0.0652,  ...,  0.0031,  0.0352,  0.0800],
        [ 0.0272, -0.0194, -0.0680,  ...,  0.0091,  0.0341,  0.0755],
        [ 0.0308, -0.0152, -0.0555,  ...,  0.0109,  0.0361,  0.0730]],
       device='cuda:0', grad_fn=<

tensor([[-0.1669, -0.0433,  0.0158,  ...,  0.1539, -0.0898,  0.0493],
        [-0.1813, -0.0990, -0.0149,  ...,  0.0903,  0.0210, -0.0477],
        [-0.1151, -0.0999,  0.0440,  ...,  0.1596,  0.0120,  0.0532],
        ...,
        [-0.2529, -0.0705, -0.0311,  ...,  0.0662, -0.0354, -0.0889],
        [-0.0467, -0.1244,  0.0072,  ...,  0.1167,  0.0253,  0.0345],
        [ 0.0888, -0.0206,  0.1046,  ...,  0.2488,  0.1124,  0.3309]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([4, 0, 2,  ..., 1, 8, 8], device='cuda:0')
tensor([[ 2.4461e-01,  1.1730e-01,  1.1488e-01,  ...,  1.1412e-01,
          2.2028e-01,  4.7261e-01],
        [-1.8716e-01, -2.3215e-02, -1.2196e-02,  ...,  4.5422e-02,
         -2.4063e-02,  1.2734e-02],
        [-2.4645e-01, -1.4418e-01,  4.4057e-04,  ...,  1.4961e-01,
         -1.3868e-01, -4.8292e-02],
        ...,
        [-2.8791e-01,  3.3607e-02, -1.2342e-02,  ...,  1.1525e-01,
         -1.7493e-01,  6.3215e-02],
        [-2.8866e-01, -7.2637e-02, -5.1535

          2.4810e+00,  2.0165e+00]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([9, 0, 5,  ..., 4, 6, 8], device='cuda:0')
tensor([[-0.2852, -0.7913,  0.3903,  ...,  0.5523, -0.4501, -1.4712],
        [ 0.1070,  0.5992,  0.0931,  ...,  0.1554, -0.0389,  0.3810],
        [-1.0658,  0.3799,  0.1545,  ...,  0.1583, -0.9027, -0.3528],
        ...,
        [ 0.6662, -0.3527,  0.4049,  ...,  0.6015,  0.0285, -0.2847],
        [ 1.3612, -0.2191,  0.4041,  ...,  0.3634,  0.6195, -0.0782],
        [ 0.0586, -0.2518,  0.4512,  ...,  0.4857, -0.3810, -0.6132]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([5, 9, 2,  ..., 0, 7, 4], device='cuda:0')
tensor([[-1.7943, -0.5434,  0.6188,  ...,  0.9422, -1.8645, -1.2202],
        [-1.7999, -0.2399,  0.5230,  ...,  0.8048, -1.7842, -1.1272],
        [-1.1186, -0.9031,  0.5957,  ...,  0.8193, -1.2815, -1.4520],
        ...,
        [-0.1392, -0.0509,  0.2992,  ...,  0.4191, -0.3562, -0.2201],
        [-0.1515,  0.5303,  0.0994,  ..., -0.

       device='cuda:0', grad_fn=<AddmmBackward>) tensor([3, 5, 9,  ..., 4, 0, 3], device='cuda:0')
tensor([[ 0.8569, -0.2904,  0.5438,  ..., -0.2900,  0.8031, -1.2034],
        [ 0.6519,  0.5731,  0.0532,  ..., -0.0461,  0.4315,  0.4980],
        [-0.8608, -0.1189,  0.5455,  ...,  0.3183, -0.9624, -0.7256],
        ...,
        [ 1.9331,  0.8053, -0.1183,  ..., -1.0147,  2.4199,  0.5603],
        [-2.6605, -0.2669,  0.9201,  ...,  1.2883, -3.0460, -0.8664],
        [ 1.0401,  0.0634,  0.1532,  ..., -0.0761,  0.8359,  0.2255]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([8, 8, 2, 4, 3, 9, 7, 2, 1, 2, 9, 8, 7, 7, 6, 3, 2, 3, 2, 9, 2, 6, 0, 4,
        9, 2, 7, 4, 7, 1, 2, 7, 1, 4, 1, 5, 3, 3, 7, 9, 2, 1, 9, 9, 7, 6, 7, 9,
        9, 7, 3, 0, 4, 7, 0, 0, 1, 9, 6, 2, 0, 3, 1, 2, 5, 4, 2, 2, 4, 9, 8, 7,
        1, 4, 2, 5, 0, 4, 6, 7, 6, 8, 3, 9, 5, 8, 1, 1, 9, 4, 9, 6, 4, 6, 2, 9,
        9, 3, 2, 2, 7, 0, 1, 6, 7, 2, 6, 1, 7, 8, 5, 2, 5, 1, 6, 8, 6, 7, 8, 4,
        3, 4, 2, 8

       device='cuda:0', grad_fn=<AddmmBackward>) tensor([6, 9, 3, 9, 8, 7, 7, 1, 6, 5, 3, 1, 3, 1, 2, 7, 1, 8, 2, 0, 9, 7, 9, 8,
        8, 6, 7, 3, 7, 1, 3, 9, 0, 9, 3, 6, 7, 2, 7, 3, 0, 5, 9, 7, 5, 5, 0, 6,
        5, 1, 8, 2, 7, 5, 9, 0, 0, 0, 8, 8, 7, 3, 7, 8, 9, 3, 7, 9, 7, 8, 7, 9,
        8, 5, 4, 8, 3, 7, 6, 3, 8, 2, 1, 9, 5, 7, 3, 9, 5, 5, 8, 7, 3, 5, 3, 5,
        9, 7, 6, 7, 3, 6, 4, 3, 9, 4, 2, 1, 9, 6, 0, 2, 6, 7, 4, 7, 9, 0, 7, 4,
        3, 5, 3, 1, 1, 2, 6, 8, 2, 1, 7, 8, 5, 9, 6, 1, 1, 5, 0, 6, 0, 9, 2, 6,
        5, 8, 9, 5, 5, 6, 2, 9, 1, 5, 8, 8, 7, 1, 7, 3, 5, 4, 9, 7, 5, 2, 9, 9,
        4, 7, 4, 1, 3, 8, 7, 9, 0, 4, 5, 7, 5, 2, 8, 7, 6, 9, 6, 9, 3, 8, 5, 6,
        6, 9, 5, 7, 8, 0, 5, 0, 7, 4, 8, 2, 5, 1, 3, 2, 2, 6, 2, 1, 7, 4, 6, 3,
        1, 3, 7, 2, 1, 3, 7, 0, 8, 4, 4, 5, 7, 9, 5, 4, 3, 9, 6, 8, 2, 3, 3, 1,
        6, 1, 7, 0, 3, 4, 2, 9, 4, 5, 8, 2, 7, 0, 9, 6, 8, 0, 8, 2, 8, 5, 7, 7,
        2, 2, 0, 0, 0, 7, 4, 1, 6, 6, 8, 8, 9, 0, 9, 0, 1, 3, 3, 0, 9, 

...

tensor([[-1.7872, -1.4918, -0.1010,  ..., -1.0881, -2.1799, -0.5253],
        [-1.6112, -1.7857,  0.8761,  ...,  0.1659, -3.5574, -2.0091],
        [ 0.2304,  0.6902, -1.4364,  ...,  1.0068, -0.5463,  1.7132],
        ...,
        [-0.4611,  1.1913, -0.9442,  ...,  0.0875, -1.3074,  2.0627],
        [-1.3555, -1.3731,  0.3406,  ...,  0.6177, -0.8668, -1.2810],
        [-1.2135, -4.1297,  2.5415,  ...,  2.4501, -3.4301, -2.7407]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([6, 3, 9,  ..., 9, 2, 5], device='cuda:0')
tensor([[-1.5843,  0.4061, -0.5575,  ..., -1.1453, -1.9108,  0.3059],
        [-1.3180, -1.2871,  0.9033,  ..., -0.0470, -0.8970, -1.4159],
        [ 0.1557,  2.2332, -2.4207,  ..., -2.2088,  0.6692,  2.5057],
        ...,
        [-1.8613, -2.2769,  1.0386,  ..., -0.3019, -1.0919, -1.4025],
        [ 1.4187, -2.7698,  1.6390,  ...,  2.1073, -1.9832, -1.8322],
        [-1.0153, -2.4462,  1.9878,  ...,  3.4837, -3.2340, -1.7105]],
       device='cuda:0', grad_fn=<

tensor([[-1.6913, -0.6883,  0.8505,  ..., -1.5840, -1.1550, -1.2773],
        [ 2.9531,  2.9698, -1.5620,  ..., -3.4110,  4.7989,  2.7632],
        [ 1.4954,  2.2521, -0.9861,  ..., -2.5456,  3.0124,  2.0067],
        ...,
        [-1.0428, -1.6956,  1.4121,  ..., -1.1173, -0.7235, -2.3327],
        [ 3.4568, -0.7274,  0.5582,  ..., -0.8074,  1.9252, -1.1134],
        [ 2.2412,  2.3906, -0.6030,  ..., -2.6016,  2.6507,  1.5391]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([3, 8, 8,  ..., 1, 0, 0], device='cuda:0')
tensor([[-1.0028, -3.4929,  2.1565,  ...,  1.4031, -2.5937, -2.8894],
        [-1.8300, -3.9778,  1.7185,  ...,  3.5457, -3.0889, -2.8073],
        [-0.3135, -1.3973,  0.7689,  ...,  0.8503, -0.8948, -1.2638],
        ...,
        [-1.0499,  1.9988, -1.5243,  ..., -0.1497, -1.4477,  2.2988],
        [-1.4917, -0.1859, -0.5645,  ...,  0.6398, -1.8116, -0.9938],
        [-1.7276, -2.0037,  1.1960,  ...,  1.2008, -2.7053, -1.8869]],
       device='cuda:0', grad_fn=<

tensor([[ 1.2646,  1.8368, -1.1676,  ...,  0.0890,  1.8837,  1.6545],
        [-0.7530,  0.5934, -0.4694,  ..., -0.8401, -0.3059,  0.6505],
        [ 1.7265,  0.0370,  0.3393,  ..., -2.1334,  2.3822,  0.0166],
        ...,
        [-0.8503,  2.1727, -2.6082,  ..., -1.9184, -0.1812,  3.3834],
        [-1.9264, -3.1944,  1.7648,  ...,  1.0368, -2.7618, -2.5358],
        [ 0.1298,  1.4391, -1.7585,  ...,  0.4604,  0.1170,  2.6757]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([9, 8, 8,  ..., 9, 6, 1], device='cuda:0')
tensor([[-1.5638, -1.0071,  1.0913,  ..., -0.8479, -1.1880, -1.0577],
        [ 0.7699,  4.0343, -1.9019,  ..., -1.7217,  1.1369,  2.1163],
        [-0.9041,  1.0243, -0.5356,  ..., -0.6565, -1.0419,  0.1611],
        ...,
        [ 1.9787,  0.2392, -0.5130,  ..., -0.4603,  1.6133,  0.7647],
        [-1.2206,  2.5590, -1.9043,  ..., -0.3113, -1.1213,  3.1808],
        [-1.0029, -1.5092,  0.7554,  ...,  0.0548, -0.9757, -0.2707]],
       device='cuda:0', grad_fn=<

tensor([[ 0.3578,  1.7730, -0.6561,  ...,  2.5981, -1.4223,  1.0252],
        [-1.7251, -2.2883,  1.1796,  ...,  0.9278, -2.7218, -1.8564],
        [-1.1662, -2.3771,  1.7911,  ...,  1.5597, -3.2691, -1.7483],
        ...,
        [-1.0362, -4.1395,  2.4238,  ...,  3.3230, -3.1932, -1.8082],
        [-1.6762, -0.3158, -0.2108,  ...,  0.3633, -1.9504,  1.1250],
        [-0.0622, -2.2887,  0.9241,  ...,  0.3259, -0.0880, -0.5041]],
       device='cuda:0', grad_fn=<AddmmBackward>) tensor([1, 2, 5,  ..., 7, 9, 4], device='cuda:0')
tensor([[-1.9159, -5.0065,  2.5139,  ...,  2.4085, -2.7876, -3.6940],
        [-1.0167, -4.7298,  3.1753,  ...,  2.2172, -3.3642, -2.7002],
        [-1.3518,  0.5255, -1.6010,  ..., -0.6012, -0.4346,  2.9205],
        ...,
        [-0.2485,  2.8394, -2.5602,  ..., -0.2196,  0.1604,  3.7828],
        [ 0.4078, -3.6399,  2.7348,  ..., -0.2023, -1.0491, -2.0530],
        [-1.6617, -1.3896,  1.0282,  ...,  0.6303, -1.7958, -1.5917]],
       device='cuda:0', grad_fn=<

KeyboardInterrupt: 

Построим график функции потерь

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(train_l)), train_l, label="train")
plt.plot(range(len(test_l)), test_l, label="test")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.show()

И, наконец, посчитаем метрики

In [ ]:
true_positive = np.zeros(10)
true_negative = np.zeros(10)
false_positive = np.zeros(10)
false_negative = np.zeros(10)
accuracy = 0
ctn = 0
for X, y in iter(test_loader):
    X = X.to(device)
    y = y.to(device)
    with torch.no_grad():
        y_pred = model(X).max(dim=1)[1]
    for i in range(10):
        for pred, real in zip(y_pred, y):
            if real == i:
                if pred == real:
                    true_positive[i] += 1
                else:
                    false_negative[i] += 1
            else:
                if pred == i:
                    false_positive[i] += 1
                else:
                    true_negative[i] += 1
            
    accuracy += torch.sum(y_pred == y).item()
    ctn += len(y)
print("Overall accuracy", accuracy / ctn)
print("Precision", true_positive / (true_positive + false_positive))
print("Recall", true_positive / (true_positive + false_negative))
print("Mean Precision", np.mean(true_positive / (true_positive + false_positive)))
print("Mean Recall", np.mean(true_positive / (true_positive + false_negative)))